In [360]:
import requests
college = input('Enter college name: ')
college=college.replace(' ', '+')

q='+site%3Awww.ratemyprofessors.com'
url = f"https://google-search3.p.rapidapi.com/api/v1/search/q={college+q}"

headers = {
	"X-User-Agent": "desktop",
	"X-Proxy-Location": "EU",
	"X-RapidAPI-Host": "google-search3.p.rapidapi.com",
	"X-RapidAPI-Key": "323948b480msha6b5bcb68958b1fp1dfdfbjsn38af2173eef9"
}

response = requests.request("GET", url, headers=headers)


In [361]:
response.text
import json
fetch = json.loads(response.text)

fetched = fetch['results']
print(fetched[0])
first_link=''
for i in range(len(fetched)):
    link=fetched[i]['additional_links'][0]['href']
    if('campusRatings' in link):
        first_link=link
        break
# first_link = fetched[0]['additional_links'][0]['href']
print(first_link)
collegeID = first_link.split('sid=')[1]
print(collegeID)


{'title': 'University of Toronto - St. George Campus - Rate My Professors', 'link': 'https://www.ratemyprofessors.com/campusRatings.jsp?sid=1484', 'description': 'This campus is full of opportunities and resources, both academic and non-academic-related. Lots of course options to choose from. Had some good profs and TAs.', 'additional_links': [{'text': 'University of Toronto - St. George Campus - Rate My Professorshttps://www.ratemyprofessors.com › ...', 'href': 'https://www.ratemyprofessors.com/campusRatings.jsp?sid=1484'}, {'text': 'En\xa0cache', 'href': 'https://webcache.googleusercontent.com/search?q=cache:gCyXYzobX7cJ:https://www.ratemyprofessors.com/campusRatings.jsp%3Fsid%3D1484+&cd=1&hl=fr&ct=clnk&gl=fr'}, {'text': 'Traduire cette page', 'href': 'https://translate.google.com/translate?hl=fr&sl=en&u=https://www.ratemyprofessors.com/campusRatings.jsp%3Fsid%3D1484&prev=search&pto=aue'}], 'cite': {'domain': 'https://www.ratemyprofessors.com › ...', 'span': ' › ...'}}
https://www.ra

In [362]:
search_url = 'https://www.ratemyprofessors.com/search/teachers?query=*&sid='
# collegeID='45'
search_url = search_url + collegeID

print(search_url)
teachers_page=requests.get(search_url)
splited=teachers_page.text.split('{"__id":"client:root:newSearch","__typename":"newSearch","teachers(after:\\"\\",first:8,query')
schoolIDNumeric = splited[1].split('})":')[0].split(':{\\"fallback\\":true,\\')[1].split(',\\"text\\":\\"\\"')[0].split('\\":\\"')[1].split('\\"')[0]
# print(schoolID)
print(schoolIDNumeric)

https://www.ratemyprofessors.com/search/teachers?query=*&sid=1484
U2Nob29sLTE0ODQ=


In [363]:
import requests
import json
# schoolID='45'
Physics_Depot='RGVwYXJ0bWVudC00NA=='
# Okay load up the headers file, parse it into a dictionary, and then add it to the request
header_json=open('headers.json')
heady = json.load(header_json)
bodi_json = open('body.json').read()
# print(bodi_json)
body = json.loads(bodi_json)
body['variables']['query']['schoolID']=schoolIDNumeric
body['variables']['schoolID']=schoolIDNumeric
body['variables']['query']['departmentID']=Physics_Depot
next_page_json=json.loads(open('next_page.json').read())

# body['variables']['query']['schoolID']=' '

heady['Referer']+=collegeID
url = 'https://www.ratemyprofessors.com/graphql'


x = requests.post(url, json = body, headers = heady)
posted =json.loads(x.text)
cursor = posted['data']['search']['teachers']['pageInfo']['endCursor']
# the cursor variable is an identifier that is used to get the next page of results
print(posted['data']['search']['teachers']['resultCount'])
# print(post['data']['search']['teachers'])

51


In [364]:
# print(post['data']['search']['teachers']['edges'][0]['cursor'])
next_page_json['variables']['count']=100
next_page_json['variables']['cursor']=cursor
next_page_json['variables']['query']['departmentID']=Physics_Depot
next_page_json['variables']['query']['schoolID']=schoolIDNumeric
professorsList=posted['data']['search']['teachers']['edges']

flipper = True
while(flipper):
    x=requests.post(url, json = next_page_json, headers = heady)
    next_page = json.loads(x.text)
    batch = (next_page['data']['search']['teachers']['edges'])
    professorsList+=batch
    cursor = next_page['data']['search']['teachers']['pageInfo']['endCursor']
    next_page_json['variables']['cursor']=cursor
    # print(next_page_json['variables']['cursor'])
    # print(next_page['data']['search']['teachers']['pageInfo'])
    print(next_page['data']['search']['teachers']['pageInfo']['hasNextPage'])
    if(next_page['data']['search']['teachers']['pageInfo']['hasNextPage']==False):
        flipper=False
        break




False


In [365]:
import pandas as pd
professorDF =pd.DataFrame(columns=['Name', 'Rating', 'Difficulty','# of Reviews','Take Again %','id'])
for profesor in professorsList:
    professor = profesor['node']
    # print(profesor['department'])
    # print(profesor['node']['department'])
    # print(profesor)
    # if(profesor['node']['department']!='Physics'):
    name = professor['firstName']+' '+professor['lastName']
    professorDF.loc[len(professorDF.index)]=[name,professor['avgRating'],professor['avgDifficulty'],professor['numRatings'],professor['wouldTakeAgainPercent'],professor['legacyId']]

    # print(name)
    # print('---')
print(len(professorsList))
print(professorsList[1])
professorDF

51
{'cursor': 'YXJyYXljb25uZWN0aW9uOjE=', 'node': {'__typename': 'Teacher', 'avgDifficulty': 2.4, 'avgRating': 3.8, 'department': 'Physics', 'firstName': 'Kimberly', 'id': 'VGVhY2hlci0xNzM2NA==', 'isSaved': False, 'lastName': 'Strong', 'legacyId': 17364, 'numRatings': 8, 'school': {'id': 'U2Nob29sLTE0ODQ=', 'name': 'University of Toronto - St. George Campus'}, 'wouldTakeAgainPercent': -1}}


,Name,Rating,Difficulty,# of Reviews,Take Again %,id
0,R Serbanescu,3.6,3,31,85.7143,10971
1,Kimberly Strong,3.8,2.4,8,-1.0000,17364
2,Salam Tawfiq,2.5,4,34,-1.0000,30788
3,Pierre Savaria,1.8,4.3,51,-1.0000,52232
4,Roberto Abraham,4.7,2.2,39,100.0000,181152
5,Jerry Mitrovica,5.0,2.6,11,-1.0000,223426
6,David Harrison,2.1,4.1,45,50.0000,258690
7,Marten van Kerkwijk,2.7,3.4,18,0.0000,294415
8,Hoi-Kwong Lo,2.3,3.3,18,-1.0000,321441
9,Hae-Young Kee,2.2,3.7,19,66.6667,380413


## Saving


In [366]:
# Make a new directory as the college name, then save the dataframe to that directory
import os
import shutil
from pathlib import Path  
file_dir = Path.cwd()

json_folder = 'Colleges'
file_path = os.path.join(file_dir, 'College Data',college.capitalize())
if not os.path.exists(file_path):
    os.makedirs(file_path)
professorDF=professorDF.sort_values(by=['# of Reviews'],ascending=False).reset_index(drop=True)


In [367]:
import numpy as np
weighted_score=np.average(a=professorDF['Rating'],weights=professorDF['# of Reviews'])
weighted_difficulty=np.average(a=professorDF['Difficulty'],weights=professorDF['# of Reviews'])
total_ratings=np.sum(professorDF['# of Reviews'])
weighted_retake=np.average(a=professorDF['Take Again %'],weights=professorDF['# of Reviews'])
print(weighted_score)
professorDF.loc[0]=['Weighted',weighted_score,weighted_difficulty,total_ratings,weighted_retake,0000]
professorDF
professorDF.to_csv(os.path.join(file_path, 'Physics Department.csv'), index=False)

3.038568129330254
